In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from math import *

%matplotlib inline

In [2]:
np.random.seed(12)
x = np.random.random(120)
y = np.random.random(120)-2
bidx = np.random.random(120)+1
askx = np.random.random(120)-0.5
bidy = np.random.random(120)+2
asky = np.random.random(120)-1.5

In [3]:
signal_exists = ['Y','N']         # exist, not exist
signal_opens  = ['O', 'C']        # open, close
signal_moves  = ['B', 'S']        # buy, sell

In [4]:
def generate_signal(priceA, priceB):
    '''Find the trading signal in one window, default as 60 days'''
    
    # test the existence of signal
    tfA = len(np.unique(np.isnan(priceA)))
    tfB = len(np.unique(np.isnan(priceB)))
            
    if  tfA == 2 or tfB == 2:
        signal_exist = signal_exists[0]
    else:
        signal_exist = signal_exists[1]

        # calculate return for two stocks
        Areturn = np.diff(priceA)/priceA[:(len(priceA)-1)]
        Breturn = np.diff(priceB)/priceB[:(len(priceB)-1)]

        # regression on return
        beta, beta0, r_value, p_value, std_err = stats.linregress(Areturn, Breturn)

        # get the residual epsilon_t
        e_t = np.array(Breturn - beta0 - beta*Areturn)

        # auxiliary process X_t
        Xt = []
        for i in range(len(Areturn)):
            Xt.append(np.sum(e_t[:i+1]))

        # regression on X_t
        length = len(Xt)
        Xt_vec = np.array(Xt)
        b, a, r_value_x, p_value_x, std_err_x = stats.linregress(Xt_vec[1:],Xt_vec[:length-1])

        # get the residual zeta_t
        z_t = Xt_vec[:length-1] - a - b*Xt_vec[1:]
        var_z = np.var(z_t)

        # calculate the s-score
        s_score = -a*sqrt(1-b**2)/((1-b)*sqrt(var_z)) + int(a/(1-b))*sqrt((1-b**2)/var_z)

        # trading signal
        if s_score < -1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[0]
        elif s_score > 1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[1]
        elif s_score < 0.75:
            signal_open = signal_opens[1]
            signal_move = signal_moves[0]
        elif s_score > -0.5:
            signal_open = signal_opens[1]
            signal_move = signal_moves[1]
        else:
            print('warning!')

        return beta,signal_exist, signal_open, signal_move

In [5]:
generate_signal(x,y)

(5.2076636313773747e-05, 'N', 'C', 'B')

In [8]:
position_dict = {}  # static variable: 'StockA': position
cash = 50000        # static variable


def build_position(position_dict, cash, pairs, data):
    '''build position for each window
    
    Default parameters
    ------------------
    shares per trade:1000
    transaction cost fee: 0.0005
    '''
    
    transaction_cost = 0.0005
    
    for i in range(60):
        for pair in pairs:
            stockA = data[data['COMNAM'] == pair[0]]
            stockB = data[data['COMNAM'] == pair[1]]
            priceA = pd.DataFrame.as_matrix(stockA['PRC'])
            priceB = pd.DataFrame.as_matrix(stockB['PRC'])
            tickerA = pair[0]
            tickerB = pair[1]
            BidA = pd.DataFrame.as_matrix(stockA['BID'])
            AskA = pd.DataFrame.as_matrix(stockA['ASK'])
            BidB = pd.DataFrame.as_matrix(stockB['BID'])
            AskB = pd.DataFrame.as_matrix(stockB['ASK'])
            current_priceA = priceA[i:i+window]
            current_priceB = priceB[i:i+window]
            result         = generate_signal(current_priceA, current_priceB)
            beta           = result[0]
            open_or_not    = result[2]
            buy_or_sell    = result[3]
            
            # for A
            if tickerA in position_dict.keys():
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash += (BidA[i+60]-transaction_cost)*position_dict[tickerA]
                        position_dict[tickerA] = 0
                    else:
                        cash -= (AskA[i+60]+transaction_cost)*position_dict[tickerA]
                        position_dict[tickerA] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] -= (cash/len(pairs))/AskB[i+60]*beta
                        cash += (asset_value/len(pairs))/AskB[i+60]*beta*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] += (cash/len(pairs))/BidB[i+60]*beta
                        cash -= (asset_value/len(pairs))/BidB[i+60]*beta*(AskA[i+60]+transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerA] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] = -(cash/len(pairs))/AskB[i+60]*beta
                        cash += abs(position_dict[tickerA])*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] = (cash/len(pairs))/BidB[i+60]*beta
                        cash -= abs(position_dict[tickerA])*(AskA[i+60]+transaction_cost)
            
            # for B
            if tickerB in position_dict.keys():
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash -= (AskB[i+60]+transaction_cost)*position_dict[tickerB]
                        position_dict[tickerB] = 0
                    else:
                        cash += (BidB[i+60]-transaction_cost)*position_dict[tickerB]
                        position_dict[tickerB] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] += (cash/len(pairs))/AskB[i+60]
                        cash -= (asset_value/len(pairs))/AskB[i+60]*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] -= (cash/len(pairs))/BidB[i+60]
                        cash += (asset_value/len(pairs))/BidB[i+60]*(BidB[i+60]-transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerB] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] = (cash/len(pairs))/AskB[i+60]
                        cash -= abs(position_dict[tickerB])*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] = (cash/len(pairs))/BidB[i+60]
                        cash += abs(position_dict[tickerB])*(BidB[i+60]-transaction_cost)
            
    return position_dict, cash